In [3]:
# ###################################
#     ----    S T A R T ----
# ###################################

# Libraries to be installed every run

!pip install moviepy
!pip install pytube 
!apt install imagemagick


# ###################################
#       -- I M P O R T A N T --
# ###################################
# imagemagick have a piece of line that needs deleting so this would read and delete the line ( rewrite file )

policypath = '/etc/ImageMagick-6/policy.xml'
try:
  with open(policypath,'r') as f:
    lines = f.readlines()
  # del policypath
except:
  print('File not found...')

# policypath = '/etc/ImageMagick-6/policy.xml'
with open(policypath,'w') as rwrite:
  for i in lines:
    if i.strip() == '<policy domain="path" rights="none" pattern="@*"/>':
      print('deleted')
    else:
      rwrite.write(i)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
imagemagick is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.9).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [4]:
import sys
import os
from pytube import YouTube # downloading videos
from moviepy.editor import * # for editing video
import subprocess # splitting videos
import re

In [30]:
def download(Link):
    cwdpath = os.getcwd()
    print(f' Your location : {cwdpath}')

    # DOWNLOAD YOUTUBE VIDEOS
    print(' Let\'s download a Video ')
    yt = YouTube(Link)
    title = yt.title
    print(f' Video Name      : {title}')
    desc = yt.description
    lenght = yt.length
    print(f' Length of video : {lenght} seconds')

    video = yt.streams.get_highest_resolution()
    vid_loc = os.path.join(cwdpath,'contents')
    path = video.download(vid_loc)

    document = {
        'Title'          : title,
        'Description'    : desc,
        'Video_Path'     : path,
        'Current_Folder' : cwdpath
    }
    with open('/content/info.txt','w') as f:
      f.write(title+'\n'+desc+'\n')

    return document

def manifest(end):
  path = '/content/contents/Nvideo.mp4'
  with open('/content/manifest.csv','w') as f:
    f.write('start_time,length,rename_to\n')
    f.write(f'0,{end},{path}')
  print('WRITING TO MANIFEST FILE')

def split(meta,end):
  #SPLIT VIDEO
  path  = meta['Video_Path'] 

  splitter = '/content/video-splitter/ffmpeg-split.py'
  if os.path.isfile(splitter) :
    print('File is available. Moving along.')
  else:
    !git clone https://github.com/c0decracker/video-splitter.git
  if end == 0:
    subprocess.run(['python', splitter ,'-f', path, '-s','59' ])
  else:
    manifest(end)
    subprocess.run(['python', splitter ,'-f', path, '-m','/content/manifest.csv' ])
    os. remove(path)

    path = '/content/contents/Nvideo.mp4'

    subprocess.run(['python', splitter ,'-f', path, '-s','59' ])

  os. remove(path) 


def changeVideo():
  lists = os.listdir('/content/contents/')
  x= 0
  paths = []
  for i in lists:
    paths.append(os.path.join('/content/contents',i))
    print(paths[x])
    x+=1
    regex = '(-\d-\d*)'
  info = '/content/info.txt'
  with open(info,'r') as f:
    p = f.readline()
  Name = p.replace('(Official Video)\n','').strip()
  path = paths[0]

  total = len(paths)
  for path in paths:
    part = re.findall('-(\w+)-',path)[0]
    try:
      print( '---------------------------------------------')
      print(f'Video Part : {part} / {total}')
      print( '---------------------------------------------')
      videoE = VideoFileClip(path, audio=True)
      w,h = moviesize = videoE.size

      txt = TextClip(f' {Name}\n Part : {part} - {total}', font='Amiri-regular',
                      color='yellow',fontsize=30);
      txt = txt.set_position(lambda t:(10,50)).set_duration(videoE.duration);
      video = CompositeVideoClip([videoE,txt.set_start(0.1).crossfadein(2)]);
      # txt = TextClip(f'Part : {part} - {total}', font='Amiri-regular',
      #                 color='yellow',fontsize=20);
      # txt = txt.set_position(lambda t:(20,80)).set_duration(video.duration);
      # video = CompositeVideoClip([video,txt.set_start(0.1).crossfadein(2)]);
      name = Name.replace(' ','')
      video.write_videofile(f'/content/contents/{name}_{part}.mp4');
      #clean up 
      os.remove(path)
    except:
      print(f'Something Wrong With Part : {part}')

def videoSavior():
  #Zip Folder and Download
  from google.colab import files
  !zip -r /content/contents/Videos.zip /content/contents/
  files.download("/content/contents/Videos.zip")

def main(end):
    Link = input(' Please Enter YouTube Link : ')
    meta = download(Link)
    #https://www.youtube.com/watch?v=d5ZFBbmvlSQ
    split(meta,end)
    changeVideo()
    videoSavior()

In [ ]:
# --------------------------------
#     --  Start Program   --
end = 210
main(end) 

## Enter Desired Video to Upload to Status / Tiktok

In [ ]:
dir = os.listdir('/content/contents/')
for i in dir:
  os.remove('/content/contents/'+i)